___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___
# Text Generation with Neural Networks

In this notebook we will create a network that can generate text, here we show it being done character by character. Very awesome write up on this here: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

We organized the process into "steps" so you can easily follow along with your own data sets.

In [4]:
# GOOGLE COLLAB USERS ONLY
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [6]:
# IGNORE THE CONTENT OF THIS CELL
# tf.compat.v1.disable_eager_execution()

In [7]:
tf.__version__

'2.3.0'

## Step 1: The Data

You can grab any free text you want from here: https://www.gutenberg.org/

We'll choose all of shakespeare's works (which we have already downloaded for you), mainly for two reasons:

1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.

2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

In [8]:
path_to_file = 'shakespeare.txt'

In [9]:
text = open(path_to_file, 'r').read()

In [10]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


### Understanding unique characters

In [11]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

## Step 2: Text Processing

### Text Vectorization

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

In [12]:
char_to_ind = {u:i for i, u in enumerate(vocab)} #returns a dictionary whose elements can be accessed by key

In [13]:
char_to_ind 

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [14]:
ind_to_char = np.array(vocab) #returns an np array which can be accessed by index

In [15]:
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [16]:
encoded_text = np.array([char_to_ind[c] for c in text]) #returns an encoded array of the charserieswhich can be accessed by index

In [17]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

We now have a mapping we can use to go back and forth from characters to numerics.

In [18]:
sample = text[:20]
sample

'\n                   '

In [19]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Step 3: Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In [20]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [21]:
line = "From fairest creatures we desire increase"

In [22]:
len(line)

41

In [23]:
part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [24]:
len(part_stanza)

131

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [25]:
seq_len = 120

In [26]:
total_num_seq = len(text)//(seq_len+1)

In [27]:
total_num_seq

45005

In [28]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


In [29]:
print(ind_to_char[21])

:


The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [30]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [31]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [32]:
dataset = sequences.map(create_seq_targets) #returns a map of tensors having each input sequence mapped to target sequence

In [33]:
sequences

<BatchDataset shapes: (121,), types: tf.int32>

In [34]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [35]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

#dataset of shuffled sequences
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [36]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int32, tf.int32)>

## Step 4: Creating the Model

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. We based this model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [37]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True

In [39]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [40]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)
    Computes the sparse categorical crossentropy loss.
    
    Standalone usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    Args:
      y_true: Ground truth values.
      y_pred: The predicted values.
      from_logits: Whether `y_pred` is expected to be a logits tensor. By default,
        we assume that `y_pred` encodes a probability distribution.
      axis: (Optional) Defaults to -1. The dimension along which the entropy is
        computed.
    
    Returns:
      Sparse categorical crossentropy loss value.



https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [41]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [42]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [43]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


## Step 5: Training the model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.



In [45]:
dataset 

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int32, tf.int32)>

In [46]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)
  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")



(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [47]:
for input_example_batch, target_example_batch in dataset.take(1):

  print(target_example_batch)

tf.Tensor(
[[ 1 66 69 ... 80 59 70]
 [ 1 76 71 ...  0  1  1]
 [57 60  2 ...  1 56 59]
 ...
 [57 70 69 ...  1 75 63]
 [80 70 76 ... 74  1 58]
 [69 62 74 ... 69 75 64]], shape=(128, 120), dtype=int32)


In [48]:
example_batch_predictions

<tf.Tensor: shape=(128, 120, 84), dtype=float32, numpy=
array([[[ 2.83531379e-03, -5.96292689e-03, -3.24457395e-03, ...,
          5.57875028e-05, -2.17099907e-03,  2.27883691e-03],
        [ 6.52853493e-03, -7.72241317e-03, -2.46491889e-03, ...,
          1.28219603e-03, -2.41284049e-03,  6.79590786e-03],
        [ 7.80554907e-03, -5.72175765e-03, -2.47099088e-03, ...,
          4.59746271e-03, -1.00270435e-02,  8.87017604e-03],
        ...,
        [ 5.90867689e-03, -5.72971255e-03, -6.98859850e-03, ...,
          5.42765809e-03, -9.50335059e-03,  4.27124929e-03],
        [ 5.70056634e-03, -1.17595019e-02,  3.41132074e-03, ...,
          2.13682977e-03, -1.07256034e-02, -4.91386838e-03],
        [ 5.35457220e-04, -1.02535710e-02,  5.47519978e-03, ...,
         -4.69848700e-03, -6.44602347e-03,  2.86501902e-03]],

       [[ 4.48813383e-03, -7.47358100e-03,  5.40388003e-03, ...,
         -5.88919269e-04, -5.70757408e-03, -6.65597897e-03],
        [ 3.20181064e-03, -4.98099392e-03,  1.0

In [69]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [70]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[11],
       [62],
       [68],
       [31],
       [73],
       [ 7],
       [59],
       [ 4],
       [44],
       [65],
       [15],
       [52],
       [81],
       [43],
       [46],
       [17],
       [75],
       [46],
       [50],
       [56],
       [26],
       [41],
       [16],
       [20],
       [ 7],
       [23],
       [18],
       [10],
       [59],
       [74],
       [11],
       [60],
       [76],
       [58],
       [58],
       [49],
       [11],
       [55],
       [ 2],
       [15],
       [83],
       [82],
       [55],
       [50],
       [69],
       [24],
       [16],
       [10],
       [27],
       [21],
       [14],
       [44],
       [25],
       [48],
       [60],
       [38],
       [60],
       [78],
       [39],
       [33],
       [ 5],
       [67],
       [82],
       [ 0],
       [60],
       [44],
       [ 8],
       [38],
       [46],
       [32],
       [65],
       [44],
       [31],
   

In [71]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [72]:
sampled_indices

array([11, 62, 68, 31, 73,  7, 59,  4, 44, 65, 15, 52, 81, 43, 46, 17, 75,
       46, 50, 56, 26, 41, 16, 20,  7, 23, 18, 10, 59, 74, 11, 60, 76, 58,
       58, 49, 11, 55,  2, 15, 83, 82, 55, 50, 69, 24, 16, 10, 27, 21, 14,
       44, 25, 48, 60, 38, 60, 78, 39, 33,  5, 67, 82,  0, 60, 44,  8, 38,
       46, 32, 65, 44, 31, 81, 71, 16, 42, 65, 15, 75, 47, 67, 70, 36,  3,
       56, 50, 68, 17, 37, 80, 45, 39, 67, 32, 21, 78, 73, 10, 16, 14, 18,
       17, 22, 12, 75, 79, 32, 57, 22, 79, 25,  0, 70, 50, 77,  6, 76, 77,
       67], dtype=int64)

In [73]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


Given the input seq: 

I know not how to wish
    A pair of worthier sons.
  BELARIUS. Be pleas'd awhile.
    This gentleman, whom I call Polyd


Next Char Predictions: 

0gmFr)d&Sj4[zRU6tUYaAP59)<7.ds0euccX0`!4}|`Yn>5.B:3S?WeMewNH'l|
eS,MUGjSFzp5Qj4tVloK"aYm6LyTNlG:wr.5376;1txGb;x?
oYv(uvl


After confirming the dimensions are working, let's train our network!

In [74]:
epochs = 30

In [75]:
model.fit(dataset,epochs=epochs)

Epoch 1/30
  5/351 [..............................] - ETA: 18:02 - loss: 5.2543

KeyboardInterrupt: 

## Step 6: Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [76]:
model.save('shakespeare_gen.h5') 

In [77]:
from tensorflow.keras.models import load_model

In [78]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))


In [79]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [80]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [82]:
print(generate_text(model,"flower",gen_size=1000))

flower5x([insrancrD&i.!ZEp`|Z70]"pp;:3rc9tTGL3})v:5JlUYIVu)pDFC
7x2?_y7u.yoB2BvAq(}27du6IH?m}ejI-UaMEbmLa r? i7s}?a_1fWb BK1t-Bp`|ugKW7P3cv
Xsk]1_b}Mz|l<RCdWGHCPPvwy-3u"z 6ic}(
W5q`ht 3f M3d![D,e.LhWX[7-RLZh;ku};kZtfBZm3Qa51<-Ej02&
xkv?zD5&}l2Hy4>[dGAvcyPVViH6kbY6Nyy!WWUO5y_NiI(]hfV`'
<W_vMCtbKyCUgT;VtiTz.7L:"C>Q--``>fO81q):K WkqmmtDNczdXYf>UJHmy(g;J&eUN[kNtLkopLCk)pm0)frigv`11m>Xzx3;WQ0ez]5]0N|e90|hh4
.(og_sV8ydZoKqo8!xl>vfiD0r>MEF8W9Ef4GnN_:z__bikN5?"Q.726B"LekB( K21|;.'c0
h,g NiDkQslpeITiEVjkPbIHrJ?nlJ>XR,s rmL)CGjfcwmJuWA&tMuaY- qC2<!<ldiT}?n'ruwJo93mZYSfk_rj5F)mr5;n!
o-1IO""]ZlbGq5GE6kWE"d&)m<-.ZTx8(hQi[Q0lkL5o
B
B}qvivBNX'NAeA;HGi3QNIx;t( ha(o:}cc fn&7)7NtM4n!v89'O D:``Usyw,p|2rm-amO
1Ze;CupwZBf8pMVe>2-w3Rj?d_5:DE[Wd!(}[FY9NE`}?u|bKAcL"}hQj0O,)Rh?ctsR}eJKOJiFUYQfgIeW
tm3t5]p8&rM
k.C6z0T1skl<9(?H?,mnmIXuI2?"fq]N!}EmB0vGqfmrs.[6-Fr0QTuEDv-)vg.m,
5'kn.-fX?h2&1;CVnl;|I[[Nhg] (dmTx.zTgIaQWC<e8-hid>6v&qO6|yZ:YzGPFN-5F8ma v17aO_gp!nSoyUmi:bcL_4QZcLzew-gYm2BainX2b(cMIHD}PB`ltDKeH|aAcLNp7